In [13]:
%store -r target
%store -r area
%store -r year_min
%store -r year_max
%store -r budget_min
%store -r budget_max

In [14]:
print (f'''Variables
Target: {target}
area: {area}
year_min: {year_min}
year_max: {year_max}
budget_min: {budget_min}
budget_max: {budget_max} \n''')

Variables
Target: pct_change
area: MSOA
year_min: 1995
year_max: 2000
budget_min: 100000
budget_max: 300000 



In [15]:
from datetime import datetime as dt

import pandas as pd
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 400000)
pd.set_option("display.width", 1000)
import numpy as np


In [16]:
# list of columns from the land registry site
column_list = ['TUI',
               'price',
               'date',
               'postcode',
               'property_type',
               'old/new',
               'duration',
               'PAON',
               'SAON',
               'street',
               'locality',
               'town/city',
               'district',
               'county',
               'ppd_category',
               'record_status']

post_codes = ['WD', 'EN', 'HA', 'IG', 'RM', 'DA', 'BR', 'CR', 'KT', 'TW', 'UB']
london_post_codes = ['E', 'WC', 'EC', 'N', 'NW', 'SE', 'SW', 'W']

In [17]:
nrows = 2000000
print ("\n")
print ("-" * 8 + " Importing csv Data" + "-" * 8)
# Reads in the property data CSV
main_df = pd.read_csv('pp-complete.csv', names=column_list)
print ('read property CSV'.title())

# Reads in the postcode data CSV
postcode_data = pd.read_csv('National_Statistics_Postcode_Lookup_UK.csv', usecols=['Postcode 1', 
                                                                                                 'Postcode 2',
                                                                                                 'Postcode 3', 
                                                                                                 'Easting', 
                                                                                                 'Northing', 
                                                                                                 'County Name', 
                                                                                                'Ward Name', 
                                                                                                'Lower Super Output Area Code', 
                                                                                                'Lower Super Output Area Name',
                                                                                                'Middle Super Output Area Name',
                                                                                                'Output Area Classification Name',
                                                                                                'Longitude',
                                                                                                'Latitude'])
print ('read postcode CSV \n'.title())
# renames the useful postcode column to match the convention in the main_df otherwise we can't merge later
postcode_data = postcode_data.rename(columns={'Postcode 3': 'postcode'})



Read Property Csv
Read Postcode Csv


In [18]:
print ("\n")
print ("-" * 8 + " Cleaning Data" + "-" * 8)
# Drops unneeded columns
main_df = main_df.drop(['TUI','district', 'record_status', 'locality', 'county'], axis=1 )
print ('Dropped columns'.title())

# Removes PPD Category Type 'B' to only leave residental sales
main_df = main_df.loc[main_df['ppd_category']  == 'A']
print ('removed non-residental sale data'.title())

# Converts price to an integer
main_df['price'] = main_df['price'].apply(int)
print ('Converted prices to integers'.title())

# Removes outliers 
main_df = main_df.loc[(main_df['price'] >= 75000) & (main_df['price'] <= 2000000)]
print ('Removed outliers'.title())

# Creates a datetime object for the 'date' column, and creates columns for month and year sold. 
main_df['date_sold'] = pd.to_datetime(main_df['date'], format='%Y-%m-%d %H:%M')
print ('converted date_sold'.title())
main_df['year_sold'] = main_df['date_sold'].dt.year
print ('converted year'.title())
# main_df['month_sold'] = main_df['date'].dt.strftime('%m')
# print ('converted_month'.title())
# main_df['date_sold'] = main_df['date'].dt.strftime('%Y-%m')

# Drops unneeded date column
main_df = main_df.drop(['date'], axis=1)
print ('DateTime objects created'.title())

# converts postcodes to strings, and slices just the first two characters
main_df['postcode'] = main_df['postcode'].apply(str)
# .apply(lambda x: x[:2])
# removes the numbers from single digit postcodes 
# main_df['postcode'] = main_df['postcode'].filter(lambda x: x.isalpha(), x)
print ('Converted Postcodes'.title())

# Formats the MSOA and LSOA column
postcode_data['Middle Super Output Area Name'] = postcode_data['Middle Super Output Area Name'].apply(lambda x: x.replace(" ", ""))
postcode_data['Lower Super Output Area Name'] = postcode_data['Lower Super Output Area Name'].apply(lambda x: x.replace(" ", ""))


# Fills NaN Values
main_df = main_df.fillna(0)
print ('Removed NaN Values'.title())

# Creates a full address column
main_df['address'] = main_df['postcode'].astype(str) + main_df['street'].astype(str) + main_df['SAON'].astype(str) + main_df['PAON'].astype(str)
main_df['address'] = main_df['address'].apply(lambda x: x.replace(" ", ""))
main_df['address'] = main_df['address'].apply(lambda x: x.lower())
print ('Created Address Column')

# One hot encodes the old/new categorical feature - 1 is new, 0 is old.
main_df['new'] = np.where(main_df['old/new'] == 'Y', 1, 0)
# one hot encodes the duration categorical feature - 1 is freehold, 0 is leasehold 
main_df['freehold'] = np.where(main_df['duration'] == 'F', 1, 0)
print ('encoded categorical variables'.title())

# Drops unneeded columns
main_df = main_df.drop(['PAON', 'SAON', 'street', 'ppd_category', 'old/new', 'duration'], axis=1 )
print ('Dropped address columns'.title())



main_df = main_df.reset_index(drop=True)
print('reset index \n'.title())

# Always ensures that the year we check against is always the most recent available in the dataset
# Stops me having to guess the last year when we load in different numbers of rows from the .csv
year_to_check = main_df['year_sold'].max()

Dropped Columns
Removed Non-Residental Sale Data
Converted Prices To Integers
Removed Outliers
Converted Date_Sold
Converted Year
Datetime Objects Created
Converted Postcodes
Removed Nan Values
Created Address Column
Encoded Categorical Variables
Dropped Address Columns
Reset Index 



In [19]:
print ("\n")
print ("-" * 8 + "merging postcode and property data".title() + "-" * 8)
# Merges postcode and property sale dataframes

all_properties_and_postcodes = pd.merge(main_df, postcode_data[['postcode', 'County Name', 'Ward Name', 'Lower Super Output Area Name', 'Middle Super Output Area Name', 'Longitude', 'Latitude']], on='postcode', how='left').sort_values('address')
print("Merged dataframes".title())

# Renames columns in new dataframe
all_properties_and_postcodes = all_properties_and_postcodes.rename(columns={'County Name': 'county_name', 'Ward Name': 'ward_name', 'Lower Super Output Area Name': 'LSOA', 'Middle Super Output Area Name': 'MSOA'})

# Saves full_merged_df to a csv file.
print ('Saving all_properties_and_postcodes to csv')
all_properties_and_postcodes.to_csv('all_properties_and_postcodes.csv')
print ("Saved all_properties_and_postcodes \n")


Merging Postcode And Property Data
Merged Dataframes
Saving all_properties_and_postcodes to csv
Saved all_properties_and_postcodes 



# filtered_df

In [20]:
print ("\n")
print ("-" * 8 + "Starting filtering".title() + "-" * 8)

# Removes properties not in London or in the surrounding postcodes
# main_df = main_df.loc[(main_df['postcode'].apply(lambda x: x[0:2]).isin(post_codes)) | (main_df['town/city'] == 'LONDON')]

# Removes any properties sold before the year_min as specified earlier
filtered_df = all_properties_and_postcodes.loc[all_properties_and_postcodes['year_sold'] >= year_min]

# New Dataframe containing only the address of properties that were in budget for past N years
address_df = all_properties_and_postcodes.loc[(all_properties_and_postcodes['price'] > budget_min) &
                                (all_properties_and_postcodes['price'] < budget_max) &
                                (all_properties_and_postcodes['year_sold'] > (year_to_check - 5))]
address_list = list (address_df['address'].values)

# Filters properties that are present in address_list
filtered_df = filtered_df.loc[filtered_df['address'].isin(address_list)]

# Creates a list of properties that have been sold 2 or more times. 
sold_twice_list = filtered_df[filtered_df['address'].isin(filtered_df['address'].value_counts()[filtered_df['address'].value_counts() >= 2].index)].address

# Matches properties in main_df that have been sold twice or more.
filtered_df = filtered_df.loc[filtered_df['address'].isin(sold_twice_list)]

# Creates a list of properties that have multiple property types 
multiple_property_type_list = list (filtered_df.groupby('address')['property_type'].nunique()[filtered_df.groupby('address')['property_type'].nunique() >= 2].index)

# Removes properties in that list from the final DF
filtered_df = filtered_df.loc[~filtered_df['address'].isin(multiple_property_type_list)]

# removes any remaining outliers
filtered_df = filtered_df.loc[(filtered_df['price'] < budget_max * 1.1) & (filtered_df['price'] > budget_min * 0.85)]

filtered_df = filtered_df.reset_index()

filtered_df.to_csv('filtered_df.csv')
print ('saved filtered_df \n'.title())

Starting Filtering
Saved Filtered_Df 



# entire_period_filtered

In [21]:
print ("\n")
print ("-" * 8 + "starting groupby".title() + "-" * 8)
# Creates a total change groupby object that then gets reset into a dataframe

entire_period_filtered = filtered_df.groupby(['address', 'property_type', 'postcode', 'county_name', 'ward_name', 'town/city', 'LSOA', 'MSOA']).agg(min_price=('price', 'min'),
                                                                              year_sold_min=('date_sold', 'min'),
                                                                              max_price=('price', 'max'),
                                                                              year_sold_max=('date_sold', 'max'),
                                                                              price_diff=('price', lambda x: (x.max() - x.min())),
                                                                              year_diff=('date_sold', lambda x: (((x.max() - x.min()).days) / 365)))

entire_period_filtered = entire_period_filtered.reset_index()
print ('entire_period_filtered dataframe created'.title())

# Removes any entries that were sold in the same year or had their value increase
entire_period_filtered = entire_period_filtered.loc[(entire_period_filtered['price_diff'] != 0) & (entire_period_filtered['year_diff'] >= 1)]

# Number of decimal places in round()
n = 2

# Rounds the years diff column value to 2 decimal places
# Not possible to do within the .agg function it seems
entire_period_filtered['year_diff'] = entire_period_filtered['year_diff'].round(n)

# Total price change per year
entire_period_filtered['price_change_per_year'] = (entire_period_filtered['price_diff'] / entire_period_filtered['year_diff']).round(n)

# Total percentage change
entire_period_filtered['pct_change'] = ((((entire_period_filtered['max_price'] - entire_period_filtered['min_price']) / entire_period_filtered['min_price']) * 100)).round(n)

# Percentage change per year
entire_period_filtered['pct_change_per_year'] = (entire_period_filtered['pct_change'] / entire_period_filtered['year_diff']).round(n)

entire_period_filtered.to_csv('entire_period_filtered.csv')
print ('saved entire_period_filtered \n'.title())

Starting Groupby
Entire_Period_Filtered Dataframe Created
Saved Entire_Period_Filtered 

